In [1]:
import os                  # for usernanme y set direcotrio
import pandas as pd
import numpy as np
import weightedcalcs as wc # ponderador o factor de expansion
from tqdm import tqdm      # controlar el tiempo en un loop ()
import warnings
warnings.filterwarnings('ignore')

## 1. Salario por hora del trabajador dependiente (Python y R)
---

#### 2019

In [65]:
# Cargando bases de datos 2019
enaho01 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2019/687-Modulo01/687-Modulo01/enaho01-2019-100.dta")
enaho34 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2019/687-Modulo34/687-Modulo34/sumaria-2019.dta")

print(enaho01.shape)
print(enaho34.shape)

(43868, 323)
(34565, 158)


In [66]:
# identificador por hogar: conglome, vivienda, hogar

merge_base_2019 = pd.merge(enaho01, enaho34,
                       on = ["conglome", "vivienda", "hogar"],
                       how = "left",
                       validate = "m:1",
                       suffixes=('', '_y'))

In [67]:
## drop variables que tenrminan en _y
index_columns = np.where( merge_base_2019.columns.str.contains('_y$', regex=True))[0]
merge_base_2019.drop(merge_base_2019.columns[index_columns], axis = 1, inplace = True)

#### 2020

In [68]:
# Cargando bases de datos 2020
enaho01 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo01/737-Modulo01/enaho01-2020-100.dta")
enaho34 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo34/737-Modulo34/sumaria-2020.dta")

In [69]:
# identificador por hogar: conglome, vivienda, hogar

merge_base_2020 = pd.merge(enaho01, enaho34,
                       on = ["conglome", "vivienda", "hogar"],
                       how = "left",
                       validate = "m:1",
                       suffixes=('', '_y'))

In [70]:
## drop variables que tenrminan en _y
index_columns = np.where( merge_base_2020.columns.str.contains('_y$', regex=True))[0]
merge_base_2020.drop(merge_base_2020.columns[index_columns], axis = 1, inplace = True)

#### juntando bases 2019 y 2020

In [71]:
# Juntando bases del 2019 y 2020
base_2019_2020 = merge_base_2019.append(merge_base_2020, ignore_index= True)  # ignore_index=True, para evitar conflictos de index
base_2019_2020

,aÑo,mes,nconglome,conglome,vivienda,hogar,ubigeo,dominio,estrato,periodo,...,ingtpu071,ingtpu08,ingtpu081,ingtpu09,ingtpu091,inghog1d1,inghog2d1,lineav_rpl,lineav,pobrezav
0,2019,10,007060,005001,007,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,10,007060,005001,017,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,10,007060,005001,028,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,10,007060,005001,040,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,10,007060,005001,050,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97286,2020,12,015122,020305,052,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,0.0,0.0,0.0,0.0,0.0,11442.000000,11442.000000,657.0,642.814026,no vulnerable
97287,2020,12,015122,020305,073,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,0.0,0.0,0.0,0.0,0.0,12083.000000,12083.000000,657.0,641.782898,no vulnerable
97288,2020,12,015122,020305,094,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97289,2020,12,015122,020305,116,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,0.0,0.0,0.0,0.0,0.0,21009.353516,21009.353516,657.0,643.160034,vulnerable no pobre


In [72]:
# extrae 2 primeros digitos

base_2019_2020['ubigeo_dep'] = base_2019_2020['ubigeo'].str[:2]

In [73]:
# convirtiendo variables a int para futuro merge

base_2019_2020[['aÑo', 'ubigeo_dep']] = base_2019_2020[['aÑo', 'ubigeo_dep']].astype(int)

#### Deflactor temporal

In [74]:
### Deflactor temporal
deflactores_base2020_new = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo34/737-Modulo34/ConstVarGasto-Metodologia actualizada/Gasto2020/Bases/deflactores_base2020_new.dta")
deflactores_base2020_new = deflactores_base2020_new[ ["dpto", "aniorec", "i00"] ]
deflactores_base2020_new.head()

,dpto,aniorec,i00
0,1.0,2001.0,0.607886
1,2.0,2001.0,0.565700
2,3.0,2001.0,0.636805
3,4.0,2001.0,0.532813
4,5.0,2001.0,0.571589


In [75]:
# identificador por hogar: conglome, vivienda, hogar

base_2019_2020 = pd.merge(base_2019_2020, deflactores_base2020_new,
                      left_on = ["ubigeo_dep", "aÑo"],
                      right_on = ["dpto", "aniorec"],
                      how = "left",
                      validate = "m:1")

In [76]:
# dividiento variables de ingreso y gasto por mieperho, 12, ld e i00

base_2019_2020['ingreso_deflact'] = base_2019_2020['inghog1d'] / (12 * base_2019_2020['mieperho'] * base_2019_2020['ld'] * base_2019_2020['i00'])
base_2019_2020['gasto_deflact']   = base_2019_2020['gashog2d'] / (12 * base_2019_2020['mieperho'] * base_2019_2020['ld'] * base_2019_2020['i00'])

In [78]:
# ordenando datos para poder compararlos con los resultados de R

base_2019_2020 = base_2019_2020.sort_values(by=['conglome', 'vivienda', 'hogar' ], axis=0, ascending=True, ignore_index=True)

base_2019_2020

,aÑo,mes,nconglome,conglome,vivienda,hogar,ubigeo,dominio,estrato,periodo,...,inghog2d1,lineav_rpl,lineav,pobrezav,ubigeo_dep,dpto,aniorec,i00,ingreso_deflact,gasto_deflact
0,2019,10,007060,005001,007,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,942.402727,818.567861
1,2019,10,007060,005001,017,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,512.260548,421.077798
2,2019,10,007060,005001,028,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,NaN,NaN
3,2019,10,007060,005001,040,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,802.508596,508.609816
4,2019,10,007060,005001,050,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,1473.884077,1121.111288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97286,2020,11,049322,020353,036,11,250103,selva,de 500 a 1 999 habitantes,3,...,NaN,NaN,NaN,NaN,25,25.0,2020.0,1.000000,NaN,NaN
97287,2020,11,049322,020353,043,11,250103,selva,de 500 a 1 999 habitantes,3,...,NaN,NaN,NaN,NaN,25,25.0,2020.0,1.000000,NaN,NaN
97288,2020,11,049322,020353,049,11,250103,selva,de 500 a 1 999 habitantes,3,...,18743.167969,657.0,661.185608,no vulnerable,25,25.0,2020.0,1.000000,1636.985232,1311.325967
97289,2020,11,049322,020353,055,11,250103,selva,de 500 a 1 999 habitantes,3,...,49633.332031,657.0,663.352417,vulnerable no pobre,25,25.0,2020.0,1.000000,558.539151,494.406453


## 2. Salario por hora del trabajador dependiente (Python y R)
---

In [79]:
# Cargando modulo 05
enaho05 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo05/737-Modulo05/enaho01a-2020-500.dta")
enaho05 = enaho05[ ["i524e1", "i538e1", "i513t", "i518"] ]

In [80]:
# reemplazando NA por ceros
enaho05 = enaho05.fillna(0)
enaho05

,i524e1,i538e1,i513t,i518
0,0.0,0.0,70.0,0.0
1,0.0,0.0,70.0,0.0
2,0.0,0.0,84.0,0.0
3,2433.0,0.0,12.0,70.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
91310,10761.0,0.0,53.0,0.0
91311,14202.0,0.0,48.0,0.0
91312,0.0,0.0,0.0,0.0
91313,7680.0,0.0,34.0,0.0


In [94]:
# generando variables necesarias
enaho05['ingreso'] = enaho05['i524e1'] + enaho05['i538e1']
enaho05['horas']   = enaho05['i513t'] + enaho05['i518']

# reemplazando NA por 0
enaho05['ingreso'] = enaho05['ingreso'].replace([''], 0)
enaho05['horas'] = enaho05['horas'].replace([''], 0)

# salario por hora
enaho05['sal_hora_depend'] = enaho05['ingreso'] / (enaho05['horas'] * 52)

# Si un salario por hora resulta 0, convertir a missing.
enaho05['sal_hora_depend'] = enaho05['sal_hora_depend'].replace([0], np.nan)

# visualizando variable calculada
enaho05[['sal_hora_depend']]

,sal_hora_depend
0,0.058909
1,0.062500
2,0.064103
3,0.069005
4,0.070251
...,...
91310,NaN
91311,NaN
91312,NaN
91313,NaN


## 3. Groupby (Python y R)
---

In [95]:
enaho02 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo02/737-Modulo02/enaho01-2020-200.dta")
enaho34 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo34/737-Modulo34/sumaria-2020.dta")

In [96]:
# identificando hogares con algun miembro mayor a 65 años
enaho02_1 = enaho02.groupby( [ "conglome", "vivienda", "hogar" ],
                            as_index = False ).agg( edad_max = ( 'p208a', max ))

enaho02_1

,conglome,vivienda,hogar,edad_max
0,005002,011,11,66.0
1,005002,023,11,62.0
2,005002,049,11,74.0
3,005002,073,11,60.0
4,005002,097,11,80.0
...,...,...,...,...
34485,020353,023,11,42.0
34486,020353,030,11,32.0
34487,020353,049,11,59.0
34488,020353,055,11,47.0


In [97]:
# inner merge
base_final_2  = pd.merge(enaho02_1, enaho34,
                       on = ["conglome", "vivienda", "hogar"],
                       how = "left",
                       validate = "m:1")

In [98]:
# Generamos variable "pobre" mediante comparación entre gasto y linea de pobreza

base_final_2['gasto_month'] = base_final_2['gashog2d'] / (12*base_final_2['mieperho'])
base_final_2["pobre"] = np.where(base_final_2["gasto_month"] < base_final_2["linea"], "pobre", "no pobre")

In [99]:
base_final_2

,conglome,vivienda,hogar,edad_max,aÑo,mes,ubigeo,dominio,estrato,mieperho,...,tipocuestionario,tipoentrevista,factor07,nconglome,sub_conglome,lineav_rpl,lineav,pobrezav,gasto_month,pobre
0,005002,011,11,66.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",4,...,cuestionario completo - antes de la emergencia...,presencial,57.675755,007061,00,657,553.702698,no vulnerable,647.939006,no pobre
1,005002,023,11,62.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",1,...,cuestionario completo - antes de la emergencia...,presencial,57.675755,007061,00,657,558.874207,no vulnerable,1993.775391,no pobre
2,005002,049,11,74.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",4,...,cuestionario completo - antes de la emergencia...,presencial,57.675755,007061,00,657,551.961121,no vulnerable,2173.834961,no pobre
3,005002,073,11,60.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",2,...,cuestionario completo - antes de la emergencia...,presencial,57.675755,007061,00,657,556.134399,no vulnerable,1173.763509,no pobre
4,005002,097,11,80.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",1,...,cuestionario completo - antes de la emergencia...,presencial,57.675755,007061,00,657,562.409973,vulnerable no pobre,328.488627,no pobre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34485,020353,023,11,42.0,2020,11,250103,selva,"de 500 a 1,999 habitantes",4,...,cuestionario completo - durante la emergencia ...,presencial,59.551132,049322,00,657,662.498047,no vulnerable,816.233805,no pobre
34486,020353,030,11,32.0,2020,11,250103,selva,"de 500 a 1,999 habitantes",3,...,cuestionario completo - durante la emergencia ...,presencial,59.551132,049322,00,657,663.237122,no vulnerable,696.493110,no pobre
34487,020353,049,11,59.0,2020,11,250103,selva,"de 500 a 1,999 habitantes",1,...,cuestionario completo - durante la emergencia ...,presencial,59.551132,049322,00,657,661.185608,no vulnerable,1251.202637,no pobre
34488,020353,055,11,47.0,2020,11,250103,selva,"de 500 a 1,999 habitantes",8,...,cuestionario completo - durante la emergencia ...,presencial,59.551132,049322,00,657,663.352417,vulnerable no pobre,471.738281,no pobre


#### Dummy mas de 65 años y pobre

In [100]:
# La estrategia es generara dummies tanto para pobreza y mayor a 65 años
# Luego se sumaran las dummies y aquellas que resulten 2 es porque cumple la condicion.
# Finalmente, se convertira el 2 en 1 para tener una dummy que cumpla con las condiciones.

base_final_2["dummy_pobre"]   = np.where(base_final_2["pobre"] == 'pobre', 1, 0)
base_final_2["dummy_mayor65"] = np.where(base_final_2["edad_max"] > 65, 1, 0)

# sumando dummies
base_final_2['dummy_suma'] = base_final_2["dummy_pobre"] + base_final_2["dummy_mayor65"]

# reemplazando 2 por 1
base_final_2['dummy_pobre_mayor65'] = np.where(base_final_2['dummy_suma'] == 2, 1, 0)


In [101]:
base_final_2

,conglome,vivienda,hogar,edad_max,aÑo,mes,ubigeo,dominio,estrato,mieperho,...,sub_conglome,lineav_rpl,lineav,pobrezav,gasto_month,pobre,dummy_pobre,dummy_mayor65,dummy_suma,dummy_pobre_mayor65
0,005002,011,11,66.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",4,...,00,657,553.702698,no vulnerable,647.939006,no pobre,0,1,1,0
1,005002,023,11,62.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",1,...,00,657,558.874207,no vulnerable,1993.775391,no pobre,0,0,0,0
2,005002,049,11,74.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",4,...,00,657,551.961121,no vulnerable,2173.834961,no pobre,0,1,1,0
3,005002,073,11,60.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",2,...,00,657,556.134399,no vulnerable,1173.763509,no pobre,0,0,0,0
4,005002,097,11,80.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",1,...,00,657,562.409973,vulnerable no pobre,328.488627,no pobre,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34485,020353,023,11,42.0,2020,11,250103,selva,"de 500 a 1,999 habitantes",4,...,00,657,662.498047,no vulnerable,816.233805,no pobre,0,0,0,0
34486,020353,030,11,32.0,2020,11,250103,selva,"de 500 a 1,999 habitantes",3,...,00,657,663.237122,no vulnerable,696.493110,no pobre,0,0,0,0
34487,020353,049,11,59.0,2020,11,250103,selva,"de 500 a 1,999 habitantes",1,...,00,657,661.185608,no vulnerable,1251.202637,no pobre,0,0,0,0
34488,020353,055,11,47.0,2020,11,250103,selva,"de 500 a 1,999 habitantes",8,...,00,657,663.352417,vulnerable no pobre,471.738281,no pobre,0,0,0,0
